<a href="https://colab.research.google.com/github/liuhaozhe6788/intro-nlp-f24/blob/master/assignment_1/problem_backprop_F24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 – Backprop

In [1]:
#@title Library Imports [do not change]

import importlib
!git clone https://www.github.com/rycolab/intro-nlp-f24.git
utils = importlib.import_module("intro-nlp-f24.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f24'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), done.


In [8]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem_i = "1" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'exp(x) - (y * 2)', 'in_vars': {'x': 2.0, 'y': -2.0}, 'output': 11.39, 'derivative': {'x': 7.39, 'y': -2.0}}
[['exp', 'x'], '-', ['y', '*', 2]] {'y': -2.0, 'x': 2.0}


In [11]:
#@title ToDo1: Building

class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)

        self.infix = infix
        self.var_id = 0  # index for available variables used
        self.graph = self.build_graph(infix)

    def build_graph(self, infix: list):
        if len(infix) == 0: return {}
        if len(infix) == 1: return {infix[0]: []}
        if len(infix) == 2:
          op, val = infix
          g = self.build_graph(val if isinstance(val, list) else [val])
          for k, v in g.items():
            if len(v) == 0:
              g[k].append([op, self.avail_vars[self.var_id]])
              g[self.avail_vars[self.var_id]] = []
              break
          self.var_id +=1
          return g
        if len(infix) == 3:
          lval, op, rval = infix
          gl = self.build_graph(lval if isinstance(lval, list) else [lval])
          gr = self.build_graph(rval if isinstance(rval, list) else [rval])
          for k, v in gl.items():
            if len(v) == 0:
              gl[k].append([op, self.avail_vars[self.var_id]])
              gl[self.avail_vars[self.var_id]] = []
              break
          for k, v in gr.items():
            if len(v) == 0:
              gr[k].append([op, self.avail_vars[self.var_id]])
              gr[self.avail_vars[self.var_id]] = []
              break
          self.var_id +=1
          g = {}
          g.update(gl)
          g.update(gr)
          return g

if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.graph)

{'x': [['exp', 'ψ']], 'ψ': [['-', 'φ']], 'φ': [], 'y': [['*', 'χ']], 'χ': [['-', 'φ']], 2: [['*', 'χ']]}


In [ ]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        pass ## ToDO: implement

    def df(self, a, b = None):
        pass ## ToDO: implement

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        pass ## ToDO: implement

    def df(self, a, b):
        pass ## ToDO: implement

class Add(Operator):

    def f(self, a, b):
        pass ## ToDO: implement

    def df(self, a, b = None):
        pass ## ToDO: implement

class Sub(Operator):

    def f(self, a, b = None):
        pass ## ToDO: implement

    def df(self, a, b = None):
        pass ## ToDO: implement

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        pass ## ToDO: implement

    def df(self, a, b=None):
        pass ## ToDO: implement

class Cos(Operator):

    def f(self, a, b=None):
        pass ## ToDO: implement

    def df(self, a, b=None):
        pass ## ToDO: implement

In [ ]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self, ):
        pass  ## ToDO: implement and set self.output
        self.output = -1

    ## backward execution____________________________

    def backward(self, ):
        pass  ## ToDO: implement and set self.derivative
        self.derivative = {}


if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

-1
{}


In [ ]:
#@title Test Function for Debugging [do not change]

utils.test_backprop(Builder, Executor, math_problem)


0: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
FAILURE output: -1 != 11.39
FAILURE derivative: {} != {'x': 7.39, 'y': -2.0}


In [ ]:
#@title Test Function for Grading [do not change]

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
FAILURE output: -1 != 1.0
FAILURE derivative: {} != {'x': 1.0, 'y': -1.0}

1: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
FAILURE output: -1 != 11.39
FAILURE derivative: {} != {'x': 7.39, 'y': -2.0}

2: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
FAILURE output: -1 != 32.0
FAILURE derivative: {} != {'x': 24.0, 'y': 8.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
FAILURE output: -1 != 0.14
FAILURE derivative: {} != {'x': -3.96, 'y': -0.99, 'z': 1.99}
